# Make NOAN NetCDF file CF-Compliant

In [ ]:
import xarray as xr
import hvplot.xarray
import pandas as pd

In [ ]:
file = '/home/jovyan/shared/noan_sm_d_1981.nc'

In [ ]:
ds = xr.open_dataset(file)

In [ ]:
ds

#### Problems to fix
* the longitude and latitude values are in global attributes instead of variables
* there is no time coordinate
* no attribute information for variables

In [ ]:
ds['lon'] = ds.Lon   
ds['lat'] = ds.Lat

attrs = ds.attrs
del attrs['Lon']
del attrs['Lat']

ds = ds.assign_attrs(attrs)

In [ ]:
dates = pd.date_range(start='1981-01-01 12:00',end='1981-12-31 12:00', freq='1d')
ds['time'] = dates
ds = ds.swap_dims({'time':'JD'})
ds = ds.rename({'JD':'time'})

In [ ]:
ds['lon'] = ds['lon'].assign_attrs({'standard_name':'longitude', 'units':'degrees_east'})
ds['lat'] = ds['lat'].assign_attrs({'standard_name':'latitude', 'units':'degrees_north'})

# Mike: see if there is are appropriate standard_names for the data variables at
# https://cfconventions.org/Data/cf-standard-names/77/build/cf-standard-name-table.html

#ds['SoilM_0_10cm'].assign_attrs({'standard_name':'change_me', 'units':'change_me'})

In [ ]:
ds

In [ ]:
%%time
fixed_nc = 'new.nc'
ds.to_netcdf(fixed_nc, mode='w')

#### Try using new fixed file

In [ ]:
ds1 = xr.open_dataset(fixed_nc)

In [ ]:
ds1

In [ ]:
var = 'SoilM_0_10cm'

In [ ]:
ds1[var].sel(time='1981-01-11 12:00').hvplot(x='lon', y='lat', geo=True, 
                                             rasterize=True, cmap='turbo_r', 
                                             tiles='OSM', alpha=0.7)